In [1]:
from torch import from_numpy, Tensor
from torch import (
    nn,
    randn,
    optim,
    zeros,
)
from os.path import join
from os import getcwd
from sklearn.model_selection import train_test_split
from numpy import array
from torch.cuda import is_available
from sklearn.preprocessing import LabelEncoder
from torch.nn.functional import one_hot

import pandas as pd
import numpy as np

import torch

In [2]:
pwd = getcwd()
datadir = join(pwd, 'data/')
file_train = join(datadir, 'train.csv')
file_test = join(datadir, 'test.csv')
file_challenge = join(datadir, 'challenge_data.csv')

In [3]:
df_train = pd.read_csv(file_train)
df_train['label'] = 1
df_train = df_train.sort_values(by=['user_id', 'challenge_sequence'])
df_test = pd.read_csv(file_test)
df_test['label'] = 0
df_test = df_test.sort_values(by=['user_id', 'challenge_sequence'])
df = pd.concat([df_train, df_test])
df = df.drop('user_sequence', axis=1)

In [4]:
challenges = pd.read_csv(file_challenge)

In [5]:
challenges.shape

(5606, 9)

In [6]:
drop_cols = [
    'publish_date',
    'author_gender',
    'author_org_ID',
    'author_ID',
    'challenge_series_ID',
]
for col in drop_cols:
    challenges = challenges.drop(col, axis=1)
challenges = challenges.fillna(0)
challenges['total_submissions'] = challenges['total_submissions'].astype(int)
challenges['category_id'] = challenges['category_id'].astype(int)
challenges = challenges.sort_values(by=['challenge_ID'])

In [7]:
challenges.shape

(5606, 4)

In [8]:
df_join = pd.merge(
    df,
    challenges,
    left_on=['challenge'],
    right_on = ['challenge_ID'],
)
df_join = df_join.drop('challenge_ID', axis=1)
df_join = df_join.sort_values(by=['user_id', 'challenge_sequence'])

In [9]:
le = LabelEncoder()
df_join['challenge'] = le.fit_transform(df_join['challenge'])

In [10]:
df_join.head()

,user_id,challenge_sequence,challenge,label,programming_language,total_submissions,category_id
0,4576,1,233,1,1,14723,29
12458,4576,2,374,1,1,20993,29
27642,4576,3,1421,1,1,43409,66
40014,4576,4,182,1,1,8897,45
52091,4576,5,451,1,1,15086,31


In [11]:
len(df_join['challenge'].value_counts())

5502

In [12]:
df_join.shape

(1301236, 7)

In [13]:
tensor_data = Tensor(df_join.values)

In [14]:
tensor_data.shape

torch.Size([1301236, 7])

In [15]:
tensor_data[:2]

tensor([[4.5760e+03, 1.0000e+00, 2.3300e+02, 1.0000e+00, 1.0000e+00, 1.4723e+04,
         2.9000e+01],
        [4.5760e+03, 2.0000e+00, 3.7400e+02, 1.0000e+00, 1.0000e+00, 2.0993e+04,
         2.9000e+01]])

In [16]:
tensor_data = tensor_data.to(torch.int64)